In [46]:
# Import the libraries
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import datetime as dt
import pandas_datareader.data as web
import yfinance as yf

# overide default yfinance download method
yf.pdr_override()
# set backend of pandas to plotly
pd.options.plotting.backend = "plotly"

In [47]:
ticker = "GS"
# Get todays date  in year, date, month format
date = dt.date.today()
type = "call"

In [48]:
def get_data(ticker: str, type: str, date: dt.date) -> pd.DataFrame:
    """ 
    This function will return a pandas dataframe of the options data for a given ticker, expirey date, and option type
    
    Parameters
    ----------
    ticker : str
        The ticker of the stock you want to get the options data for
    date : datetime.date
        The date you want to get the options data for
    type : str
    """
    # Get options data
    try:    
        options = web.YahooOptions(ticker)
        options.headers = {'User-Agent': 'Firefox'}
    except:
        print("Error: Could not get options data for ticker: ", ticker)
        return None

    # Get the call data
    expirey_dates = options._get_expiry_dates()
    # Get the closest expirey date to today
    closest_date = min(expirey_dates, key=lambda x: abs(x - date))
    time_delta = closest_date - date

    # Get the call data
    if type == "call":
        data = options.get_call_data(expiry=closest_date)
    elif type == "put":
        data = options.get_put_data(expiry=closest_date)

    return data

In [50]:
class OptionsData:

    def __init__(self, ticker: str, expirey: dt.date, option_type: str):
        self.ticker = ticker
        self.expirey = expirey
        self.option_type = option_type
        self.data = self.get_closest_data()
    
    def get_closest_data(self) -> pd.DataFrame:
        """ 
        This function will return a pandas dataframe of the options data for a given ticker, 
        expirey date, and option type. It will return the closest expirey date to the given date
        
        Parameters
        ----------
        ticker : str
            The ticker of the stock you want to get the options data for
        date : datetime.date
            The date you want to get the options data for
        type : str
            The type of option you want to get the data for. Must be either 'call' or 'put'
        
        Returns
        -------
        data : pd.DataFrame
            A pandas dataframe of the options data for the given ticker, expirey date, and option type
        """
        try:    
            options = web.YahooOptions(self.ticker)
            options.headers = {'User-Agent': 'Firefox'}
        except:
            print("Error: Could not get options data for ticker: ", self.ticker)
            return None
        
        # Get 
        
        data = web.get_data_yahoo(self.ticker, self.expirey, self.expirey)
        options = yf.Ticker(self.ticker).option_chain(self.expirey)
        
        # Get the call data
        if type == "call":
            data = options.get_call_data(expiry=closest_date)
        elif type == "put":
            data = options.get_put_data(expiry=closest_date)

        else:
            raise ValueError("option_type must be either 'call' or 'put'")
        
        # merge the options data with the stock data
        data = data.merge(options, left_index=True, right_index=True)
        return data

In [52]:
yf.Ticker(ticker).option_chain(date)


ValueError: Expiration `2023-03-25` cannot be found. Available expiration are: [2023-03-31, 2023-04-06, 2023-04-14, 2023-04-21, 2023-04-28, 2023-05-19, 2023-06-16, 2023-07-21, 2023-09-15, 2023-10-20, 2023-12-15, 2024-01-19, 2024-06-21, 2025-01-17, 2025-06-20, 2025-12-19]

In [39]:
expirey_dates

[datetime.date(2023, 3, 31),
 datetime.date(2023, 4, 6),
 datetime.date(2023, 4, 14),
 datetime.date(2023, 4, 21),
 datetime.date(2023, 4, 28),
 datetime.date(2023, 5, 19),
 datetime.date(2023, 6, 16),
 datetime.date(2023, 7, 21),
 datetime.date(2023, 9, 15),
 datetime.date(2023, 10, 20),
 datetime.date(2023, 12, 15),
 datetime.date(2024, 1, 19),
 datetime.date(2024, 6, 21),
 datetime.date(2025, 1, 17),
 datetime.date(2025, 6, 20),
 datetime.date(2025, 12, 19)]